In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import operator
import spacy
import sklearn as sk
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

%matplotlib inline

In [2]:
df = pd.read_csv('Resources/tweet_data.csv')

In [15]:
df['tidy_tweet'] = df['tidy_tweet'].astype(str)
df['tweet'] = df['tweet'].astype(str)

In [3]:
df.head()

,target,id,date,user,tweet,tidy_tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",Awww that bummer You shoulda got David Carr Th...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,scotthamilton,is upset that he can't update his Facebook by ...,upset that can update his Facebook texting and...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,mattycus,@Kenichan I dived many times for the ball. Man...,dived many times for the ball Managed save The...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,ElleCTF,my whole body feels itchy and like its on fire,whole body feels itchy and like its fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,Karoli,"@nationwideclass no, it's not behaving at all....",not behaving all mad why here because can see ...


In [4]:
import string
punctuations = string.punctuation
# Creating a Spacy Parser
from spacy.lang.en import English
parser = English()

In [22]:
import spacy
from  spacy.lang.en.stop_words import STOP_WORDS
#nlp = spacy.load('en')

# To build a list of stop words for filtering
stopwords = list(STOP_WORDS)

In [5]:
def my_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    return mytokens

In [6]:
# ML Packages
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

In [7]:
#Custom transformer using spaCy 
class predictors(TransformerMixin):
    def transform(self, X, **transform_params):
        return [clean_text(text) for text in X]
    def fit(self, X, y, **fit_params):
        return self
    def get_params(self, deep=True):
        return {}

# Basic function to clean the text 
def clean_text(text):     
    return text.strip().lower()

In [9]:
# Vectorization
vectorizer = CountVectorizer(tokenizer = my_tokenizer, ngram_range=(1,1)) 
classifier = LinearSVC()

In [10]:
# Using Tfidf
tfvectorizer = TfidfVectorizer(tokenizer = my_tokenizer)

In [16]:
from sklearn.model_selection import train_test_split

# Features and Labels
X = df['tidy_tweet']
ylabels = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2, random_state=42)

In [23]:
# Create the  pipeline to clean, tokenize, vectorize, and classify using"Count Vectorizor"
pipe_countvect = Pipeline([("cleaner", predictors()),
                 ('vectorizer', vectorizer),
                 ('classifier', classifier)])
# Fit our data
pipe_countvect.fit(X_train,y_train)

C:\Users\Benji\Anaconda3\envs\PythonData\lib\site-packages\sklearn\svm\_base.py:977: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


Pipeline(steps=[('cleaner', <__main__.predictors object at 0x00000199019AC208>),
                ('vectorizer',
                 CountVectorizer(tokenizer=<function my_tokenizer at 0x00000199000E87B8>)),
                ('classifier', LinearSVC())])

In [25]:
import joblib

joblib.dump(pipe_countvect, 'spacy_pipeline.pkl')

['spacy_pipeline.pkl']

In [26]:
del pipe_countvect